In [1]:
from MOT import MOT
import numpy as np
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
import open3d as op3
# from open3d.web_visualizer import draw
from Utils import *
import pandas as pd
from scipy.sparse import csr_matrix

In [131]:
a = 6378137
b = 6356752.31414
e1=(a**2-b**2)/(a**2) #First eccentricity of the ellipsoid
e2=(a**2-b**2)/(b**2) #second eccentricity of the ellipsoid

In [132]:
A = np.concatenate([np.array([
    [-19.09431575,-25.32061849,5.78213465],
    [-9.88950927,-0.2624204,-0.92349226],
    [21.52768583,17.57006876,-2.26844151],
    [23.43061836,-1.17874425,-1.36626699]
]),np.ones(4).reshape(-1,1)],axis = 1)

temp = np.array([
    [36.01755845,	-115.0832427,	686.1517033],
    [36.01752893,	-115.0829601,	686.1517033],
    [36.01725356,	-115.0827685,	686.1517033],
    [36.01722806,	-115.0829583,	686.1517033]
])
temp[:,[0,1]] = temp[:,[0,1]]*np.pi/180
temp[:,2] = temp[:,2]/3.2808

In [142]:
N = a/np.sqrt((1 - e1 * np.sin(temp[:,0])**2)) 
B = np.concatenate([
    ((N + temp[:,2]) * np.cos(temp[:,0]) * np.cos(temp[:,1])).reshape(-1,1),
  ((N + temp[:,2]) * np.cos(temp[:,0]) * np.sin(temp[:,1])).reshape(-1,1),
  ((N*(1 - e1) + temp[:,2]) * np.sin(temp[:,0])).reshape(-1,1)
 ],axis = 1)
B = np.concatenate([B,np.ones(4).reshape(-1,1)],axis = 1)
T = np.linalg.inv((A.T).dot(A)).dot(A.T.dot(B))

In [143]:
test_traj = pd.read_csv('../2020-3-1-0-0-0-BF1-CL1-Traj(0-18000frames).csv')
xyz  = np.array(test_traj.loc[:,['Coord_x','Coord_y','Coord_z']])
xyz1 = np.concatenate([xyz,np.ones(len(xyz)).reshape(-1,1)],axis = 1)
XYZ1 = xyz1.dot(T)


In [163]:
lon = (np.arctan(XYZ1[:,1]/XYZ1[:,0])-np.pi)*180/np.pi
theta = np.arctan(a * XYZ1[:,2]/(b * np.sqrt(XYZ1[:,0]**2 + XYZ1[:,1]**2)))
lat = np.arctan((XYZ1[:,2] + e2*b*np.sin(theta)**3)/(np.sqrt(XYZ1[:,0]**2 + XYZ1[:,1]**2) - e1*a*np.cos(theta)**3))
evel = XYZ1[:,2]/np.sin(lat) - (a/np.sqrt((1 - e1 * np.sin(lat)**2)))*(1 - e1)
# XYZ1[:,2]/np.sin(lat) - a/np.sqrt((1 - e1 * np.sin(lat*np.pi/180)**2)) 
lat = lat*180/np.pi

In [175]:
pd.DataFrame({
    'Lat':lat,
    'Lon':lon,
    'Evel':evel
}).to_csv(r'./result.csv',index=False)

In [165]:
val = pd.read_csv('../test.csv')


In [166]:
re = pd.DataFrame({
    'Lat':lat,
    'Lon':lon,
    'Evel':evel
})

In [3]:
TrajPath = r'.\OutputFile\OutputTrajs'
TrajLis = os.listdir(TrajPath)

FileNotFoundError: [Errno 2] No such file or directory: '.\\OutputFile\\OutputTrajs'

In [ ]:
TPM = np.zeros((400,400))

In [37]:
TPMs = [np.zeros((400*400,400*400)) for i in range(7)]

MemoryError: Unable to allocate 191. GiB for an array with shape (160000, 160000) and data type float64

In [30]:
# rule out the estimated values by KF
TestTraj = np.array(pd.read_csv(os.path.join(TrajPath,TrajLis[0])))[:,[0,1]][:-8]
BinX,BinY = np.arange(-200,201),np.arange(-200,201)
XIndex,YIndex = np.digitize(TestTraj[:,0],BinX),np.digitize(TestTraj[:,1],BinY)
for tpm in TPMs:
    

In [ ]:
np.zeros((200,200,))

In [20]:
TestTraj.iloc[:-8]

,X_Coord_mea,Y_Coord_mea,X_Len_mea,Y_Len_mea,Z_Len_mea,X_Coord_est,Y_Coord_est,X_Len_est,Y_Len_est,Z_Len_est,X_Vel_est,Y_Vel_est,X_Acc_est,Y_Acc_est
0,-11.880474,-24.119835,5.745061,3.441625,1.998897,-11.880474,-24.119835,5.745061,3.441625,1.998897,0.000000,0.000000,0.000000,0.000000
1,-11.368456,-24.197860,5.664705,3.208388,1.987152,-11.786089,-24.134218,5.668514,3.219442,1.987708,0.062645,-0.009546,0.020882,-0.003182
2,-11.024649,-24.375524,5.496608,3.371619,2.002513,-11.413091,-24.245639,5.512613,3.340198,2.000077,0.291764,-0.082358,0.092412,-0.027100
3,-10.769361,-24.353723,4.778183,2.999653,1.992186,-10.886457,-24.349060,4.904718,3.086393,1.995657,0.496676,-0.113938,0.125370,-0.028413
4,-10.471005,-24.214727,4.444730,2.422467,2.000417,-10.421378,-24.305243,4.464018,2.588782,1.999413,0.573218,-0.053293,0.113066,-0.005970
5,-10.226033,-24.236790,3.776464,2.431757,1.967476,-10.069100,-24.281851,3.874649,2.388622,1.979858,0.557386,-0.022253,0.084361,0.002272
6,-9.911183,-24.461892,3.335502,2.340459,1.958870,-9.739566,-24.400137,3.358502,2.261529,1.964841,0.526517,-0.061446,0.061200,-0.006062
7,-9.634861,-24.415833,2.605300,2.102447,1.946846,-9.446976,-24.436091,2.712378,2.083689,1.950491,0.482500,-0.056163,0.041797,-0.003970
8,-9.321026,-24.570280,1.969080,1.984450,2.114373,-9.155233,-24.536879,2.051161,1.944257,2.041065,0.445053,-0.076098,0.028184,-0.006713
9,-8.997133,-24.676813,1.517818,1.815612,1.647589,-8.858574,-24.648977,1.494903,1.795908,1.823872,0.415482,-0.094414,0.018795,-0.008599


In [ ]:
TPM = np.zeros((200,200))

In [ ]:
np.digitize()